# Data Generation

In [1]:
from datetime import datetime, timedelta
import json
import random

In [2]:
# mason, plumber, electrician, carpenter, painter
trades = ['MA','PL', 'EL','CA', 'PA']
today = datetime.utcnow()
sim_start = today +timedelta(weeks=10, days=4)
sim_start.strftime('%Y-%m-%dT%H:%M:%SZ')

'2020-02-16T09:43:11Z'

In [3]:
def number_of_rooms(max_rooms=6):
    # length = random.randrange(3, max_rooms, 1)
    # return range(0,length)
    return 6

In [4]:
def moc_deadlines(trades=[]):
    offset = random.randrange(0, 7, 1)
    start = datetime.utcnow() - timedelta(days=offset)
    return [
    (start + timedelta(weeks=index)).strftime('%Y-%m-%dT%H:%M:%SZ') 
     for index,trade in enumerate(trades)
    ]

In [5]:
def moc_room_pipeline(trades=[], deadlines=[]):
    return [
        {'tradesmen': trade, 'deadline': deadline} for trade, deadline in zip(trades,deadlines)
    ] 
        

In [6]:
def subset(trades=[]):
    start = random.randrange(0, int(len(trades)*0.75)+1, 1)
    return trades[start:]

In [7]:
def generate_building_data(floors=5):
    building_data = {}
    for floor in range(10):
        room_data = {}
        for room in number_of_rooms():
            trade_subset = subset(trades)
            pipeline = moc_room_pipeline(trade_subset, moc_deadlines(trade_subset))
            room_data[f'room_{floor}{room}'] = pipeline
        building_data[f'floor_{floor}'] = room_data
        
    return building_data

In [8]:
def deadlines(trades=[], offset = 0):
    start = datetime.utcnow()
    end = datetime.utcnow() + timedelta(weeks=1)
    return [
        {
            'trade': trade,
            'start': (start + timedelta(weeks=index + offset)).strftime('%Y-%m-%dT%H:%M:%SZ'), 
            'end': (end + timedelta(weeks=index+ offset)).strftime('%Y-%m-%dT%H:%M:%SZ'),
            'done': sim_start >= end + timedelta(weeks=index+ offset)
            
        }
        for index,trade in enumerate(trades)
    ]

In [9]:
site_data = {}
zones = 0
for building in [
    ('A1', 6, {'x': 0, 'y': 0, 'z': 0, 'rotation': 0}), 
    ('A2', 5, {'x': 15, 'y': 0, 'z': 5, 'rotation': 0}), 
    ('A3', 8, {'x': 15, 'y': 0, 'z': -15, 'rotation': 45})
]:
    name, floors, coords = building
    building_data = {}
    building = {}
    for floor in range(floors):
        floor_data = {}
        for zone in range(2):
            pipeline = deadlines(trades, zones)
            zone_data = {
                'pipeline': pipeline,
                'zone': zone,
                'zoneId': zones
            }
            floor_data[f'zone_{zone}'] = zone_data
            zones += 1
        building_data[f'floor_{floor}'] = floor_data
        building = {
            'coords': coords,
            'data': building_data
        }
    site_data[name] = building
    zones = 0

In [10]:
with open('buildingDataV2.json', 'w') as json_file:
    json.dump(site_data, json_file)

In [11]:
site_data

{'A1': {'coords': {'x': 0, 'y': 0, 'z': 0, 'rotation': 0},
  'data': {'floor_0': {'zone_0': {'pipeline': [{'trade': 'MA',
       'start': '2019-12-04T09:43:11Z',
       'end': '2019-12-11T09:43:11Z',
       'done': True},
      {'trade': 'PL',
       'start': '2019-12-11T09:43:11Z',
       'end': '2019-12-18T09:43:11Z',
       'done': True},
      {'trade': 'EL',
       'start': '2019-12-18T09:43:11Z',
       'end': '2019-12-25T09:43:11Z',
       'done': True},
      {'trade': 'CA',
       'start': '2019-12-25T09:43:11Z',
       'end': '2020-01-01T09:43:11Z',
       'done': True},
      {'trade': 'PA',
       'start': '2020-01-01T09:43:11Z',
       'end': '2020-01-08T09:43:11Z',
       'done': True}],
     'zone': 0,
     'zoneId': 0},
    'zone_1': {'pipeline': [{'trade': 'MA',
       'start': '2019-12-11T09:43:11Z',
       'end': '2019-12-18T09:43:11Z',
       'done': True},
      {'trade': 'PL',
       'start': '2019-12-18T09:43:11Z',
       'end': '2019-12-25T09:43:11Z',
       'do